In [1]:
import librosa
import os
import numpy as np
import sys
from dtw import dtw
from numpy.linalg import norm
from numpy import array
import pyaudio
import wave

import heapq

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



**|Heap| = 80**  
**n_mfcc = 20**  
**DTWSeq = mfcc**  
**None**

In [2]:
def initialCorpus(path):
    # 音乐库位置
    audioList = os.listdir(path)

    raw_audioList = {}
    beat_database = {}

    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav'):
            # 读入一维音频序列
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save('beatDatabase_mfcc_20.npy', beat_database)
    
    return beat_database

In [3]:
def readCorpus(path):
    
    # 读入音乐节奏数据库
    all_data = np.load(path, allow_pickle=True)
    beat_database = all_data.item()
    
    return beat_database

In [4]:
def updateCorpus(path, dbpath):
    
    # 音乐库位置
    audioList = os.listdir(path)
    
    # 已保存序列的文件
    raw_db = readCorpus(dbPath)
    raw_files = raw_db.keys()
    
    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav') and audioName not in raw_files:
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save(dbpath, beat_database)

In [5]:
def normlize(data):
    n_mean = np.mean(data, axis=0)
    n_std  = np.std(data, axis=0)
    
    norm_data = np.divide(np.subtract(data, n_mean), n_std)
    return norm_data

In [6]:
def voiceCompare_quick(dbPath, tPath):
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # 读入要识别的录音
    y, sr = librosa.load(tPath)

    # 识别录音的节奏序列
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    beat_frames = librosa.feature.delta(beat_frames,mode ='nearest')
    x = array(beat_frames).reshape(-1, 1)

    # 将待识别的录音序列与语料库中语音逐一做DTW对比
    compare_result = {}
    
    for songID in beat_database.keys():
        y = beat_database[songID]
        y = array(y).reshape(-1, 1)
        
        dist = dtw(x, y).distance
        # print('两段话的差异程度为： ', songID.split("\\")[1], ": ", dist)
        
        compare_result[songID] = dist

    matched_song = min(compare_result, key=compare_result.get)
    print("最接近的录音是：", matched_song)

In [8]:
from sklearn import preprocessing

def voiceCompare(dbPath, tPath):
    # 最大检索数
    aimNum = 80
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # ==== 读入要识别的录音 ====
    y, sr = librosa.load(tPath)

    # 提取录音的 MFCC 特征
    x = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20).T  # n1 * 20
    lenx = len(x)

    # ==== 将待识别的录音序列与语料库中语音逐一做DTW对比 ====
    
    # heap for [dist, 时间段，文件名]
    heap = []
    heapq.heapify(heap)  
    
    for songID in beat_database.keys():
        # 取出文件名对应的 mfcc 序列
        y = beat_database[songID].T
        
        leny = len(y) # n2 * 20 
        print(leny)

        for tp in range(0, leny - lenx):
            # *加速* 设定距离上限
            full = False  # 堆是否已满
            dist_UB = -10000  # DTW 距离上限
            overBound = False  # 是否过限
            
            if (len(heap) >= aimNum):
                full = True
                dist_UB = -heap[0][0]  # heap top (biggest) DTW dist as UB  
                
            # 计算 DTW(y[tp : tp + lenx])
            total_dist = 0
            
            for i in range(0, lenx):
                # DTW dist
                total_dist += dtw(x[i], y[tp + i], distance_only=False).distance
                
                # *加速* 超过上限直接取消
                if (full and total_dist > dist_UB):
                    overBound = True
                    break
            
            # *加速* 超过上限
            if (overBound):
                continue
            
            # 入栈
            tupleY = (-total_dist, tp, songID) # dtw 距离加负数转为大根堆
            
            heapq.heappush(heap, tupleY)
            if (len(heap) > aimNum):
                heapq.heappop(heap)
            
            print(tupleY)
            
        # end for
        
        # 处理同名短间隔问题
        
        
    return heap

In [9]:
def getTimePoint_dense(dbPath, tPath, vheap):
    res_num = 20 # 定义取出前 res_num 位的结果作为识别结果
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()
    
    # 得到要识别的录音时长
    tTime = librosa.get_duration(filename=tPath)
    
    # 提取前 res_num 个相似的片段并输出对应时间段
    similar_n = heapq.nlargest(res_num, vheap)
    
    print("开始输出相似片段：")
    
    for i in range(0, res_num):
        music_name = similar_n[i][2]  # 录音文件名
        music_time = librosa.get_duration(filename=music_name)  # 录音时长
        
        music_pos = similar_n[i][1]  # 时间段所在帧数
        music_all = len(beat_database[music_name][0])  # 录音总帧数

        frag_st = music_time / music_all * music_pos  # 时间段起点
        frag_en = frag_st + tTime  # 时间段终点
        
        # print(music_name, music_time, music_pos, music_all, frag_st)
        # print("相似度第", i + 1, "位的为文件 ", music_name, "的 ", '%.2f' % frag_st, "到", '%.2f' % frag_en, "秒")
        
        print(music_name, ",", '%.2f' % frag_st, "秒,", '%.2f' % frag_en, "秒")


In [14]:
def getTimePoint(dbPath, tPath, vheap):
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()
    
    # 得到要识别的录音时长
    tTime = librosa.get_duration(filename=tPath)
    
    heapq.nlargest(20, vheap)
    
    # ====== 对 vheap 进行去重 ======
    # 取出文件名
    name_set = set()
    for tp in vheap:
        name_set.add(tp[2])
    # print(name_set)
    
    # 合并下标差小于5的片段
    sheap = []
    for name in name_set:
        # 按下标排序
        nList = [x for x in vheap if x[2] == name]
        sortL = sorted(nList, key=lambda t:t[1])
        
        # 去重
        for tp in sortL:
            if len(sheap) < 1 or sheap[-1][2] != name or abs(sheap[-1][1] - tp[1]) > 5:
                sheap.append(tp)
            else:  
                if (sheap[-1][0] < tp[0]): 
                    sheap[-1] = tp  # 保留距离较小项

    # print(sheap)
    # 提取相似片段并输出对应时间段
#     heapq.heapify(sheap)
#     if (len(sheap) > 10):
#         sheap = heapq.nlargest (10, sheap)
        
    similar_n = sheap
        
    print("开始输出相似片段：")
    
    for i in range(0, len(sheap)):
        music_name = similar_n[i][2]  # 录音文件名
        music_time = librosa.get_duration(filename=music_name)  # 录音时长
        
        music_pos = similar_n[i][1]  # 时间段所在帧数
        music_all = len(beat_database[music_name][0])  # 录音总帧数
        
        frag_st = music_time / music_all * music_pos  # 时间段起点
        frag_en = frag_st + tTime  # 时间段终点
        # print(music_name, music_time, music_pos, "/", music_all, "=", frag_st)
        
        # print(music_name, music_time, music_pos, music_all, frag_st)
        # print("相似度第", i + 1, "位的为文件 ", music_name, "的 ", '%.2f' % frag_st, "到", '%.2f' % frag_en, "秒")
        
        print(music_name, ",", '%.2f' % frag_st, "秒,", '%.2f' % frag_en, "秒")

In [11]:
# 语料库路径
corpus_path = './corpus'

# 数据表路径
dbPath = './beatDatabase_mfcc_20.npy';

# test file path
# testPath = './input/00415250-前5s.wav'
# testPath = './input/00429126-53s_60s.wav'
testPath = './input/00430105-hou5s.wav'

In [13]:
# 1 初始化语料序列库
# beatDB = initialCorpus(corpus_path)

# 2 更新语料库中新音乐文件的序列
# updateCorpus(corpus_path, dbPath)

# 3 读入语料序列库
# beat_database = readCorpus(dbPath)
testPath = './input/00430105-hou5s.wav'
vheap = voiceCompare(dbPath, testPath)

3876
(-1552.395266573545, 0, './corpus\\00415250.wav')
(-1573.9153801972461, 1, './corpus\\00415250.wav')
(-1590.4417497285358, 2, './corpus\\00415250.wav')
(-1606.343303674268, 3, './corpus\\00415250.wav')
(-1617.040728059732, 4, './corpus\\00415250.wav')
(-1623.5257282499406, 5, './corpus\\00415250.wav')
(-1641.580820257328, 6, './corpus\\00415250.wav')
(-1657.9537389136567, 7, './corpus\\00415250.wav')
(-1668.6474987713336, 8, './corpus\\00415250.wav')
(-1691.8583879553662, 9, './corpus\\00415250.wav')
(-1709.761838433118, 10, './corpus\\00415250.wav')
(-1740.8867901854865, 11, './corpus\\00415250.wav')
(-1758.3295897136159, 12, './corpus\\00415250.wav')
(-1779.7628959873873, 13, './corpus\\00415250.wav')
(-1805.2332557277005, 14, './corpus\\00415250.wav')
(-1828.3066111823027, 15, './corpus\\00415250.wav')
(-1841.5009139474896, 16, './corpus\\00415250.wav')
(-1848.2566539900326, 17, './corpus\\00415250.wav')
(-1859.9433391462262, 18, './corpus\\00415250.wav')
(-1867.2553972634669, 

(-1481.8394642039384, 658, './corpus\\00415250.wav')
(-1473.3872183617386, 659, './corpus\\00415250.wav')
(-1477.1145144032962, 660, './corpus\\00415250.wav')
(-1469.52610157269, 661, './corpus\\00415250.wav')
(-1456.5905465847063, 662, './corpus\\00415250.wav')
(-1451.065459388792, 663, './corpus\\00415250.wav')
(-1451.562275689812, 664, './corpus\\00415250.wav')
(-1453.8889216268617, 665, './corpus\\00415250.wav')
(-1459.9238681746829, 666, './corpus\\00415250.wav')
(-1478.9981415805123, 667, './corpus\\00415250.wav')
(-1501.6192561544194, 668, './corpus\\00415250.wav')
(-1505.378413483082, 669, './corpus\\00415250.wav')
(-1504.3197622237967, 695, './corpus\\00415250.wav')
(-1500.1697337339465, 697, './corpus\\00415250.wav')
(-1491.6621207412572, 698, './corpus\\00415250.wav')
(-1494.6413725950006, 704, './corpus\\00415250.wav')
(-1481.6684262984963, 705, './corpus\\00415250.wav')
(-1457.859010512755, 706, './corpus\\00415250.wav')
(-1448.5637816496783, 707, './corpus\\00415250.wav')

(-1395.8822731899745, 1711, './corpus\\00415250.wav')
(-1395.651766736777, 1712, './corpus\\00415250.wav')
(-1395.9839706018283, 1713, './corpus\\00415250.wav')
(-1396.0630696981862, 1714, './corpus\\00415250.wav')
(-1396.0173698484614, 1733, './corpus\\00415250.wav')
(-1395.8341447978928, 1734, './corpus\\00415250.wav')
(-1396.1536367654517, 1735, './corpus\\00415250.wav')
(-1396.215665568383, 1736, './corpus\\00415250.wav')
(-1396.215665568383, 1737, './corpus\\00415250.wav')
(-1396.215665568383, 1738, './corpus\\00415250.wav')
(-1396.215665568383, 1739, './corpus\\00415250.wav')
(-1396.215665568383, 1740, './corpus\\00415250.wav')
(-1396.215665568383, 1741, './corpus\\00415250.wav')
(-1396.215665568383, 1742, './corpus\\00415250.wav')
(-1396.215665568383, 1743, './corpus\\00415250.wav')
(-1396.215665568383, 1744, './corpus\\00415250.wav')
(-1396.215665568383, 1745, './corpus\\00415250.wav')
(-1396.215665568383, 1746, './corpus\\00415250.wav')
(-1396.215665568383, 1747, './corpus\\00

10164
7150
(-1371.1264477989062, 911, './corpus\\00430105.wav')


In [14]:
getTimePoint(dbPath, testPath, vheap)

开始输出相似片段：
./corpus\00415250.wav , 26.35 秒, 28.42 秒
./corpus\00415250.wav , 76.56 秒, 78.62 秒
./corpus\00430105.wav , 21.15 秒, 23.22 秒
./corpus\00429239.wav , 12.24 秒, 14.31 秒
./corpus\00429239.wav , 102.35 秒, 104.42 秒
./corpus\00429239.wav , 207.21 秒, 209.28 秒
./corpus\00429239.wav , 207.35 秒, 209.42 秒
./corpus\00429239.wav , 226.48 秒, 228.55 秒
./corpus\00429239.wav , 226.62 秒, 228.69 秒
./corpus\00429126.wav , 80.27 秒, 82.34 秒
./corpus\00429126.wav , 94.73 秒, 96.80 秒


In [ ]:
inPara = sys.argv

if (len(inPara) < 2):
    print("请输入待识别录音文件路径！")
else:
    if (len(inPara) > 2):
        print("给定语料库路径为:", sys.argv[2])
        corpus_path = sys.argv[2]
    else:
        print("默认语料库路径为：", corpus_path)
    
    if (len(inPara) > 3):
        print("给定数据表路径为:", sys.argv[3])
        dbPath = sys.argv[3]
    else:
        print("默认数据表路径为：", dbPath)
    
    testPath = sys.argv[1]
    vheap = voiceCompare(dbPath, testPath)
    getTimePoint(dbPath, testPath, vheap)
  

In [12]:
testPath2 = './input/00430105-hou5s.wav'
vheap2 = voiceCompare(dbPath, testPath2)
getTimePoint(dbPath, testPath2, vheap2)

3876
(-69058.64055424929, 0, './corpus\\00415250.wav')
(-69007.90790617466, 1, './corpus\\00415250.wav')
(-68886.29880622774, 2, './corpus\\00415250.wav')
(-68927.43599152565, 3, './corpus\\00415250.wav')
(-68852.82537150383, 4, './corpus\\00415250.wav')
(-68820.19707170874, 5, './corpus\\00415250.wav')
(-68905.08852321655, 6, './corpus\\00415250.wav')
(-69093.69332294166, 7, './corpus\\00415250.wav')
(-69355.13443147391, 8, './corpus\\00415250.wav')
(-69591.15640944242, 9, './corpus\\00415250.wav')
(-69842.97855297476, 10, './corpus\\00415250.wav')
(-70025.3648135215, 11, './corpus\\00415250.wav')
(-69971.77840039879, 12, './corpus\\00415250.wav')
(-69984.93101556599, 13, './corpus\\00415250.wav')
(-70063.28121867031, 14, './corpus\\00415250.wav')
(-69883.95995979756, 15, './corpus\\00415250.wav')
(-69724.70703686774, 16, './corpus\\00415250.wav')
(-69744.7396940589, 17, './corpus\\00415250.wav')
(-69611.49443986267, 18, './corpus\\00415250.wav')
(-69426.78030530363, 19, './corpus\\00

(-62579.8765944615, 179, './corpus\\00415250.wav')
(-62513.47637068853, 180, './corpus\\00415250.wav')
(-62386.34843316302, 181, './corpus\\00415250.wav')
(-61898.2698851414, 182, './corpus\\00415250.wav')
(-61640.69622113928, 183, './corpus\\00415250.wav')
(-61349.69530282542, 184, './corpus\\00415250.wav')
(-61175.124672386795, 185, './corpus\\00415250.wav')
(-61479.13406311348, 186, './corpus\\00415250.wav')
(-62332.42665247992, 187, './corpus\\00415250.wav')
(-62544.02465419844, 188, './corpus\\00415250.wav')
(-62321.24906050041, 189, './corpus\\00415250.wav')
(-61732.08414521441, 190, './corpus\\00415250.wav')
(-61044.88439165428, 191, './corpus\\00415250.wav')
(-60475.23534397781, 192, './corpus\\00415250.wav')
(-59883.55426083878, 193, './corpus\\00415250.wav')
(-59056.364764321595, 194, './corpus\\00415250.wav')
(-58606.44668842852, 195, './corpus\\00415250.wav')
(-58541.049644075334, 196, './corpus\\00415250.wav')
(-58282.2445919998, 197, './corpus\\00415250.wav')
(-57833.4248

(-58165.47617889941, 1428, './corpus\\00415250.wav')
(-58012.355227038264, 1429, './corpus\\00415250.wav')
(-57958.2479108274, 1430, './corpus\\00415250.wav')
(-58056.5937833786, 1431, './corpus\\00415250.wav')
(-57924.12191584706, 1432, './corpus\\00415250.wav')
(-57610.00890390575, 1433, './corpus\\00415250.wav')
(-57871.092617064714, 1434, './corpus\\00415250.wav')
(-57875.53633847833, 1437, './corpus\\00415250.wav')
(-57219.97665990889, 1438, './corpus\\00415250.wav')
(-57060.21085469425, 1439, './corpus\\00415250.wav')
(-57331.93019582331, 1440, './corpus\\00415250.wav')
(-57840.301907137036, 1441, './corpus\\00415250.wav')
(-57748.85308007896, 1442, './corpus\\00415250.wav')
(-57972.31197933853, 1443, './corpus\\00415250.wav')
(-57769.63666033745, 1447, './corpus\\00415250.wav')
(-57915.879327729344, 1448, './corpus\\00415250.wav')
(-57467.75347143412, 1460, './corpus\\00415250.wav')
(-56776.635505869985, 1461, './corpus\\00415250.wav')
(-56041.32376310229, 1462, './corpus\\00415

(-54531.92870532721, 2297, './corpus\\00415250.wav')
(-54642.32139532268, 2298, './corpus\\00415250.wav')
(-54763.19396109134, 2299, './corpus\\00415250.wav')
(-55226.38370300084, 2300, './corpus\\00415250.wav')
(-55253.834380693734, 2301, './corpus\\00415250.wav')
(-55361.96472269297, 2302, './corpus\\00415250.wav')
(-55433.42645935714, 2304, './corpus\\00415250.wav')
(-55283.440123051405, 2305, './corpus\\00415250.wav')
(-54930.54091259837, 2306, './corpus\\00415250.wav')
(-54788.37647873908, 2307, './corpus\\00415250.wav')
(-54386.87611079961, 2308, './corpus\\00415250.wav')
(-54481.81178288907, 2309, './corpus\\00415250.wav')
(-55169.516321875155, 2310, './corpus\\00415250.wav')
8786
(-54634.79015810788, 1886, './corpus\\00429126.wav')
(-55029.68313795328, 1887, './corpus\\00429126.wav')
(-55016.99280548096, 2019, './corpus\\00429126.wav')
(-55178.21151761711, 2020, './corpus\\00429126.wav')
(-55155.437465682626, 2023, './corpus\\00429126.wav')
(-54553.6542237699, 2024, './corpus\\

(-52560.74372828007, 6633, './corpus\\00429126.wav')
(-52245.98341667652, 6634, './corpus\\00429126.wav')
(-52312.14281925559, 6635, './corpus\\00429126.wav')
(-52456.291721478105, 6636, './corpus\\00429126.wav')
(-52331.172943234444, 6637, './corpus\\00429126.wav')
(-52481.47136107087, 6638, './corpus\\00429126.wav')
(-52466.45791131258, 6639, './corpus\\00429126.wav')
(-52529.93367622793, 6640, './corpus\\00429126.wav')
(-52368.806727051735, 6641, './corpus\\00429126.wav')
(-52169.625662088394, 6642, './corpus\\00429126.wav')
(-51808.67413055897, 6643, './corpus\\00429126.wav')
(-52074.56504243612, 6644, './corpus\\00429126.wav')
(-52414.06271213293, 6645, './corpus\\00429126.wav')
(-52610.58220284432, 6906, './corpus\\00429126.wav')
12231
(-52647.382900848985, 11325, './corpus\\00429239.wav')
(-52602.832440160215, 11326, './corpus\\00429239.wav')
10164
7150
(-52446.468037055805, 1464, './corpus\\00430105.wav')
(-52095.570339361206, 1465, './corpus\\00430105.wav')
(-52087.41597047634

In [15]:
getTimePoint(dbPath, testPath2, vheap2)

开始输出相似片段：
./corpus\00430105.wav , 64.71 秒, 66.77 秒
./corpus\00430105.wav , 75.57 秒, 77.64 秒
./corpus\00430105.wav , 75.71 秒, 77.78 秒
./corpus\00430105.wav , 75.97 秒, 78.03 秒
./corpus\00430105.wav , 119.71 秒, 121.77 秒
./corpus\00430105.wav , 121.01 秒, 123.08 秒
./corpus\00430105.wav , 140.02 秒, 142.09 秒
./corpus\00430105.wav , 140.16 秒, 142.23 秒
./corpus\00429126.wav , 88.25 秒, 90.32 秒
./corpus\00429126.wav , 88.39 秒, 90.46 秒
./corpus\00429126.wav , 88.56 秒, 90.63 秒
./corpus\00429126.wav , 89.21 秒, 91.28 秒
./corpus\00429126.wav , 137.22 秒, 139.29 秒
./corpus\00429126.wav , 145.88 秒, 147.95 秒
./corpus\00429126.wav , 146.02 秒, 148.09 秒
./corpus\00429126.wav , 154.24 秒, 156.31 秒
